In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import torch 
import torch.nn as nn

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from utils import *
from models import *
from tqdm import tqdm

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dmin = 2
dmax = 5
m = 8
t = 100
n = 50000
SNRmin = -10
SNRmax = -10
lamda = 0.2
distance = 0.1
coherent = False

lr = 1e-3
wd = 1e-9
nbEpoches = 200
batchSize = 128

array = ULA(m, lamda)
array.build_array(distance)
array.build_array_manifold()

In [ ]:
observations, angles, labels = generate_data(n, t, dmin, dmax, SNRmin, SNRmax, array, coherent)

x_train, x_valid, theta_train, theta_valid, label_train, label_valid = train_test_split(observations, angles, labels, test_size=0.2)
x_train, x_test, theta_train, theta_test, label_train, label_test = train_test_split(x_train, theta_train, label_train, test_size=0.2)

train_set = DATASET(x_train, theta_train, label_train)
valid_set = DATASET(x_valid, theta_valid, label_valid)
test_set = DATASET(x_test, theta_test, label_test)

train_loader = DataLoader(train_set, batch_size=batchSize, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False)

In [ ]:
model = ECNet(m=m, dmin=dmin, dmax=dmax,
              nbLayers=10, nbNeurons=10, device=dev)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

loss_func = nn.CrossEntropyLoss()

Loss, Val = [], []

bestVal = 0.0

for i in tqdm(range(nbEpoches)):

    running_loss = 0.0

    for data in train_loader:
        
        X, label_true = data[0].to(dev), data[2].to(dev)
        optimizer.zero_grad()
        label_pred = model(X)
        loss = loss_func(label_pred, label_true)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    Loss.append(running_loss/len(train_loader))

    with torch.no_grad():

        running_acc = 0.0

        for data in valid_loader:
            
            X, label_true = data[0].to(dev), data[2].to(dev)
            label_pred = model(X)
            acc = (torch.argmax(label_pred, dim=1) == torch.argmax(label_true, dim=1)).float().mean()
            running_acc += acc.item()
        
        Val.append(running_acc/len(valid_loader))

        if Val[-1] > bestVal:
            bestVal = Val[-1]
            torch.save(model.state_dict(), "ecnet.pth")
            count = 0
        else:
            count += 1

        if count == 20:
            model.load_state_dict(torch.load("ecnet.pth", weights_only=True))


    print("Iteration = {}, Loss = {}, Accuracy = {}".format(i, Loss[-1], Val[-1]))

In [ ]:
model = ECNet(m=m, dmin=dmin, dmax=dmax,
              nbLayers=10, nbNeurons=10, device=dev)
model.load_state_dict(torch.load("ecnet.pth", weights_only=True))

running_acc = 0.0

for data in test_loader:

    X, theta_true, label_true = data[0].to(dev), data[1].to(dev), data[2].to(dev)
    label_pred = model(X)
    acc = (torch.argmax(label_pred, dim=1) == torch.argmax(label_true, dim=1)).float().mean()
    running_acc += acc.item()

print("Accuracy of ECNet is {}%".format(running_acc/len(test_loader)*100))